In [5]:
import numpy as np
import scipy.optimize
import scipy.stats
from IPython.display import display
import pandas as pd

np.set_printoptions(precision=3, suppress=True)
pd.set_option('precision', 3)

ImportError: cannot import name 'NUMPY_MKL'

# Init Robot

In [20]:
# load UR10 model for testing
model = np.array([
    0, 0, 0, 118,
    1.5707963267949, 0, 3.14159265358979, 0,
    0, 612.7, 0, 0,
    0, 571.6, 0, 163.9,
    - 1.5707963267949, 0, 0, 115.7,
    1.5707963267949, 0, 3.14159265358979, 92.2
]).reshape((-1, 4))
robot = pybot.Robot(model)

df = pd.DataFrame(robot.robot_model, columns=('alpha', 'a', 'theta', 'd'))
display(df)

,alpha,a,theta,d
0,0.000,0.0,0.000,118.0
1,1.571,0.0,3.142,0.0
2,0.000,612.7,0.000,0.0
3,0.000,571.6,0.000,163.9
4,-1.571,0.0,0.000,115.7
5,1.571,0.0,3.142,92.2


# Init Tool

In [15]:
nominal_tool_xyz = [100, 100, 50]
real_tool_xyz = [99.123, 100.987, 51.192]

# "Real" Measures

In [16]:
# simulate real measures (this would actually be externally measured data)
robot.tool.tcp_xyz(real_tool_xyz)

joints = []
torques = []
positions = []
for i in range(100):
    robot.random_joints()
    joints.append(robot.joint_angles)
    torques.append([0] * robot.num_dof())
    positions.append(robot.fk()[:-1, -1])

# Calibrate Tool

## Compute Initial Errors

In [22]:
# use nominal tool as seed for calibration
robot.tool.tcp_xyz(nominal_tool_xyz)

initial_errors = pybot.calibration.compute_absolute_errors(robot=robot,
                                                           joints=joints,
                                                           torques=torques,
                                                           positions=positions)
print('Initial tool:\n{}'.format(robot.tool.tcp))
print('Initial errors:\n{}'.format(scipy.stats.describe(initial_errors)))

Initial tool:
[[   1.    0.    0.  100.]
 [   0.    1.    0.  100.]
 [   0.    0.    1.   50.]
 [   0.    0.    0.    1.]]
Initial errors:
DescribeResult(nobs=100, minmax=(1.7788091522137663, 1.7788091522140106), mean=1.778809152213918, variance=1.0768927472033373e-27, skewness=-0.7036938138240161, kurtosis=4.553317223827894)


## Calibrate Using Least Squares

In [23]:
def calibration_fitness_func(optimization_vector,
                             optimization_mask,
                             robot,
                             joints,
                             torques,
                             positions):
    # validate input
    assert len(joints) == len(torques)
    assert len(joints) == len(positions)

    # make sure optimization_vector is a list
    if isinstance(optimization_vector, np.ndarray):
        optimization_vector = optimization_vector.tolist()

    robot.apply_optimization_vector(optimization_vector, optimization_mask)

    errors = pybot.calibration.compute_absolute_errors(robot, joints, torques, positions)

    return errors

In [26]:
# set calibration mask
tool_mask = [True] * 6
tool_mask[3] = False
tool_mask[4] = False
tool_mask[5] = False

optimization_mask = robot.generate_optimization_mask(tool_mask=tool_mask)
optimization_vector = robot.generate_optimization_vector(optimization_mask)

# calibrate
optimize_result = scipy.optimize.leastsq(func=calibration_fitness_func,
                                         x0=optimization_vector,
                                         args=(
                                             optimization_mask,
                                             robot,
                                             joints,
                                             torques,
                                             positions))

# process results
calibrated_tool = optimize_result[0]
print('Calibrated tool:\n{}'.format(calibrated_tool))
print('Calibration error:\n{}'.format(calibrated_tool-real_tool_xyz))

Calibrated tool:
[  99.123  100.987   51.192]
Calibration error:
[ 0. -0.  0.]
